In [ ]:
# csv 转化为 json

import pandas as pd
import json

items = {}
item_df = pd.read_csv('Beauty2014/Beauty2014.item.csv')
for row in item_df.itertuples():
    item_id = getattr(row, "item_id")
    title = getattr(row, "title")
    discription = getattr(row, "description")
    title = "" if pd.isnull(title) else title
    discription = "" if pd.isnull(discription) else discription
    items[item_id] = {"title": title, "discription": discription}

with open('Beauty2014/Beauty2014.item.json', 'w') as f:
    json.dump(items, f, ensure_ascii=False, indent=4)

inters = {}
inter_df = pd.read_csv("Beauty2014/Beauty2014.inter.csv")
for row in inter_df.itertuples():
    user_id = getattr(row, "user_id")
    item_id = getattr(row, "item_id")
    if user_id not in inters:
        inters[user_id] = []
    inters[user_id].append(item_id)

with open('Beauty2014/Beauty2014.inter.json', 'w') as f:
    json.dump(inters, f, ensure_ascii=False, indent=4)


In [ ]:
# 统计数据集的信息
import json

inters = json.load(open("Beauty2014/Beauty2014.inter.json", "r"))

user_count = {}
item_count = {}

for user_id, items in inters.items():
    for item_id in items:
        if item_id not in item_count:
            item_count[item_id] = 0
        item_count[item_id] += 1
    user_count[user_id] = len(items)

train_target_count = []
valid_target_count = []
test_target_count = []

for user_id, items in inters.items():
    for index in range(1, len(items)-2):
        train_target_count.append(item_count[items[index]])
    valid_target_count.append(item_count[items[-2]])
    test_target_count.append(item_count[items[-1]])
print(f"item_avg_count: {sum(item_count.values())/len(item_count)}")
print(f"train_target_count: {sum(train_target_count)/len(train_target_count)}")
print(f"valid_target_count: {sum(valid_target_count)/len(valid_target_count)}")
print(f"test_target_count: {sum(test_target_count)/len(test_target_count)}")

In [ ]:
# 将sasrec.pth 转化为 pt
import torch
import os
path = "" # dir path
os.chdir(path)  # 切换目录
sasrec = torch.load("save/Beauty2014/SASRec-d_model_32-train_batch_size_256-lr_0.01-wd_0.01.pth", weights_only=False)
emb = sasrec.item_emb.weight
torch.save(emb, "data/Beauty2014/Beauty2014-cf_emb.pt")

In [12]:
import torch
import os
path = "/home/liuchang/RecModels" # dir path
os.chdir(path)  # 切换目录
from torch.utils.data import DataLoader
from data.dataset import SeqRecDataset
import torch.nn as nn
def prepare_input(
    his_seqs,
    next_items,
    indice_matrix,
):
    # his_seqs [batch_size, max_len]
    # next_items [batch_size]

    encoder_input_ids = indice_matrix(his_seqs) # [batch_size, max_len, codebook_num]
    decoder_input_ids = indice_matrix(next_items) # [batch_size, codebook_num]

    # rescale his_seqs and next_items
    encoder_shift = (
        torch.tensor([2 + i * 256 for i in range(encoder_input_ids.shape[-1])])
        .reshape(1, 1, encoder_input_ids.shape[-1])
        .to(encoder_input_ids.device)
    )
    shift = decoder_shift = (
        torch.tensor([2 + i * 256 for i in range(decoder_input_ids.shape[-1])])
        .reshape(1, decoder_input_ids.shape[-1])
        .to(decoder_input_ids.device)
    )
    encoder_input_ids = (encoder_input_ids + encoder_shift).long() # [batch_size, max_len, codebook_num]
    decoder_input_ids = (decoder_input_ids + decoder_shift).long() # [batch_size, codebook_num]


    # write pad to encoder_input_ids
    pad_indices = (his_seqs == 0).long().nonzero(as_tuple=True)
    encoder_input_ids[pad_indices[0], pad_indices[1]] = torch.zeros(encoder_input_ids.shape[-1], dtype=torch.long).to(encoder_input_ids.device)
    encoder_input_ids = encoder_input_ids.reshape(encoder_input_ids.shape[0], -1) # [batch_size, max_len*codebook_num]

    # add eos to encoder_input_ids
    encoder_input_ids = torch.cat(
        [
            encoder_input_ids, 
            torch.ones((encoder_input_ids.shape[0], 1), dtype=torch.long).to(encoder_input_ids.device)
        ], 
        dim=-1
    ) # [batch_size, max_len*codebook_num + 1]

    # add eos to labels
    labels = torch.cat(
        [
            decoder_input_ids,
            torch.ones((decoder_input_ids.shape[0], 1), dtype=torch.long).to(encoder_input_ids.device),
        ],
        dim=-1
    ) # [batch_size, codebook_num + 1]

    # add decoder_input_token to decoder_input_ids
    decoder_input_ids = torch.cat(
        [
            torch.zeros((decoder_input_ids.shape[0], 1), dtype=torch.long).to(encoder_input_ids.device),
            decoder_input_ids
        ],
        dim=-1
    ) # [batch_size, 1 + codebook_num]

    encoder_mask = (encoder_input_ids != 0).int() # [batch_size, max_len*codebook_num + 1]

    return encoder_input_ids, encoder_mask, decoder_input_ids, labels, shift



data_path = "./data"
dataset_name = "Beauty2014"
max_len = 20
seed = 1024
num_workers = 4
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

rqvae = torch.load("save/Beauty2014/TIGER/rqvae-tiger-lr_0.001-wd_0.0001-best.pth", weights_only=False).to(device)
t54rec = torch.load("save/Beauty2014/TIGER/TIGER-rqvae_lr_0.001-rqvae_wd_0.0001-rqvae_select_position_best-t54rec_lr_0.0005-t54rec_wd_0.01-t54rec_epochs_200-scheduler_type_cosine-warmup_ratio_0.01-t54rec_train_batch_size_256-t54rec_early_stop_step_20-beam_size_20.pth", weights_only=False).to(device)
rqvae.update_labels()
rqvae_indices = rqvae.get_all_indices()
indice_matrix = nn.Embedding(rqvae.item_emb.weight.shape[0], 4).to(device)
indice_matrix.weight.data.copy_(rqvae_indices)
codebook1_labels_list = []
codebook2_labels_list = []
codebook3_labels_list = []
codebook4_labels_list = []
for idx, vq in enumerate(rqvae.rq.codebooks):
    labels = vq.labels
    for i in range(10):
        indices = [j for j in range(len(labels)) if labels[j] == i]
        if idx == 0:
            codebook1_labels_list.append([j + 2 + idx * 256 for j in indices])
        elif idx == 1:
            codebook2_labels_list.append([j + 2 + idx * 256 for j in indices])
        elif idx == 2:
            codebook3_labels_list.append([j + 2 + idx * 256 for j in indices])
        elif idx == 3:
            codebook4_labels_list.append([j + 2 + idx * 256 for j in indices])

dataset = SeqRecDataset(
    data_root_path=data_path, 
    dataset=dataset_name, 
    max_len=max_len, 
    mode="test", 
    seed=seed
)

dataloader = DataLoader(
    dataset, 
    batch_size=64, 
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

all_probs_spe = []
all_probs_codebook1 = []
all_probs_codebook2 = []
all_probs_codebook3 = []
all_probs_codebook4 = []
all_probs_codebook_cluster = []

with torch.no_grad():
    for step, batch in enumerate(dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        his_seqs = batch["his_seqs"]
        next_items = batch["next_items"]
        (
            encoder_input_ids, 
            encoder_mask, 
            decoder_input_ids, 
            labels,
            shift,
        ) = prepare_input(
            his_seqs,
            next_items,
            indice_matrix
        )

        logits = t54rec._forward(
            input_ids=encoder_input_ids,
            attention_mask=encoder_mask,
            decoder_input_ids=decoder_input_ids,
            labels=labels,
            return_dict=False
        )[1][:, :-1, :] # [batch_size, 4, vocab_size]
        target_cluster = []
        for i in range(logits.shape[0]):
            clusters = []
            for j in range(4):
                clusters.append(rqvae.rq.codebooks[j].labels[decoder_input_ids[i, j + 1].cpu().numpy() - 2 - j * 256])
            target_cluster.append(torch.tensor(clusters))
        target_cluster = torch.stack(target_cluster, dim=0).unsqueeze(-1) # [batch_size, 4, 1]

        probs = torch.softmax(logits, dim=-1) # [batch_size, 4, vocab_size]

        # 码表间概率分布
        probs_spe = probs[:, :, :2].sum(dim=-1) # [batch_size, 4]
        probs_codebook1 = probs[:, :, 2: 258].sum(dim=-1) # [batch_size, 4]
        probs_codebook2 = probs[:, :, 258: 514].sum(dim=-1) # [batch_size, 4]
        probs_codebook3 = probs[:, :, 514: 770].sum(dim=-1) # [batch_size, 4]
        probs_codebook4 = probs[:, :, 770:].sum(dim=-1) # [batch_size, 4]

        # 码表内聚簇概率分布
        probs_codebook_cluster = []
        for i in range(probs.shape[0]):
            probs_codebook1_cluster_temp = []
            probs_codebook2_cluster_temp = []
            probs_codebook3_cluster_temp = []
            probs_codebook4_cluster_temp = []
            for j in range(4):
                if j == 0:
                    for k in range(10):
                        probs_codebook1_cluster_temp.append(probs[i, j, codebook1_labels_list[k]].sum())
                elif j == 1:
                    for k in range(10):
                        probs_codebook2_cluster_temp.append(probs[i, j, codebook2_labels_list[k]].sum())
                elif j == 2:
                    for k in range(10):
                        probs_codebook3_cluster_temp.append(probs[i, j, codebook3_labels_list[k]].sum())
                elif j == 3:
                    for k in range(10):
                        probs_codebook4_cluster_temp.append(probs[i, j, codebook4_labels_list[k]].sum())
            probs_codebook1_cluster_temp = torch.tensor(probs_codebook1_cluster_temp).reshape(1, 10)
            probs_codebook2_cluster_temp = torch.tensor(probs_codebook2_cluster_temp).reshape(1, 10)
            probs_codebook3_cluster_temp = torch.tensor(probs_codebook3_cluster_temp).reshape(1, 10)
            probs_codebook4_cluster_temp = torch.tensor(probs_codebook4_cluster_temp).reshape(1, 10)
            probs_codebook_cluster.append(torch.cat([probs_codebook1_cluster_temp, probs_codebook2_cluster_temp, probs_codebook3_cluster_temp, probs_codebook4_cluster_temp], dim=0))
        probs_codebook_cluster = torch.stack(probs_codebook_cluster, dim=0).gather(-1, target_cluster) # [batch_size, 4]

        all_probs_spe.append(probs_spe)
        all_probs_codebook1.append(probs_codebook1)
        all_probs_codebook2.append(probs_codebook2)
        all_probs_codebook3.append(probs_codebook3)
        all_probs_codebook4.append(probs_codebook4)
        all_probs_codebook_cluster.append(probs_codebook_cluster)
    all_probs_spe = torch.cat(all_probs_spe, dim=0).mean(dim=0)
    all_probs_codebook1 = torch.cat(all_probs_codebook1, dim=0).mean(dim=0)
    all_probs_codebook2 = torch.cat(all_probs_codebook2, dim=0).mean(dim=0)
    all_probs_codebook3 = torch.cat(all_probs_codebook3, dim=0).mean(dim=0)
    all_probs_codebook4 = torch.cat(all_probs_codebook4, dim=0).mean(dim=0)
    all_probs_codebook_cluster = torch.cat(all_probs_codebook_cluster, dim=0).mean(dim=0)
    print(all_probs_spe)
    print(all_probs_codebook1)
    print(all_probs_codebook2)
    print(all_probs_codebook3)
    print(all_probs_codebook4)
    print(all_probs_codebook_cluster)

tensor([2.3159e-04, 4.3609e-05, 1.4472e-05, 4.7719e-06], device='cuda:0')
tensor([9.8870e-01, 2.9377e-03, 2.6375e-04, 6.3479e-05], device='cuda:0')
tensor([4.0028e-03, 9.8898e-01, 4.8369e-04, 9.3770e-05], device='cuda:0')
tensor([4.2089e-03, 4.8651e-03, 9.9879e-01, 2.0790e-04], device='cuda:0')
tensor([2.8552e-03, 3.1775e-03, 4.4549e-04, 9.9963e-01], device='cuda:0')
tensor([[0.2872],
        [0.1807],
        [0.7915],
        [0.9398]])
